In [3]:
from download_delgado.delgado_datasets import DownloadAndConvertDelgadoDatasets
from mleap.data import Data
from mleap.estimators.estimators import instantiate_default_estimators
from mleap.experiments import Orchestrator
from mleap.analyze_results import AnalyseResults

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### Step 1: Load the datasets

In [4]:
#load datasets as pandas DataFrame
#each dataset needs to have metadata attached to it containing the following keys:
# class_name: (string) name of column containing the column name in which the labels are stored
# dataset_name: (string) name of the dataset
delgado = DownloadAndConvertDelgadoDatasets()
datasets, metadata = delgado.download_and_extract_datasets(verbose = False) #think about storing in hdd

Error: Dataset Delgado_data/molec-biol-protein-second has a different number of arff files


### Step 2: Save the datasets in HDF5 database

In [5]:
# files io object needs to be used in test orch
# returned object needs to be integrated in code
data = Data()
data.pandas_to_db(save_loc_hdf5='delgado_datasets/', datasets=datasets, 
                  dts_metadata=metadata, save_loc_hdd='data/delgado.hdf5') #return files io object
input_io = data.open_hdf5('data/delgado.hdf5', mode='r')
out_io = data.open_hdf5('data/classification.hdf5', mode='a')

### Step 3: Instantiate models and Test Orchestrator

In [4]:
instantiated_models = instantiate_default_estimators(estimators=['Generalized Linear Models'])

test_o = TestOrchestrator(hdf5_input_io=input_io, hdf5_output_io=out_io)

### Step 4: Split datasets

In [7]:
dts_names_list, dts_names_list_full_path = data.list_datasets(hdf5_io=input_io, hdf5_group='delgado_datasets/')
split_dts_list = data.split_datasets(hdf5_in=input_io, hdf5_out=out_io, dataset_paths=dts_names_list_full_path)

2018-03-06 23:29:38,452 [MainThread  ] [WARNI]  Skipping abalone as test/train split already exists in output h5 file.
2018-03-06 23:29:38,465 [MainThread  ] [WARNI]  Skipping acute_inflammation as test/train split already exists in output h5 file.
2018-03-06 23:29:38,476 [MainThread  ] [WARNI]  Skipping acute_nephritis as test/train split already exists in output h5 file.
2018-03-06 23:29:38,490 [MainThread  ] [WARNI]  Skipping adult as test/train split already exists in output h5 file.
2018-03-06 23:29:38,507 [MainThread  ] [WARNI]  Skipping annealing as test/train split already exists in output h5 file.
2018-03-06 23:29:38,527 [MainThread  ] [WARNI]  Skipping arrhythmia as test/train split already exists in output h5 file.
2018-03-06 23:29:38,545 [MainThread  ] [WARNI]  Skipping audiology_std as test/train split already exists in output h5 file.
2018-03-06 23:29:38,572 [MainThread  ] [WARNI]  Skipping balance_scale as test/train split already exists in output h5 file.
2018-03-06 23:

2018-03-06 23:29:39,717 [MainThread  ] [WARNI]  Skipping nursery as test/train split already exists in output h5 file.
2018-03-06 23:29:39,736 [MainThread  ] [WARNI]  Skipping oocytes_merluccius_nucleus_4d as test/train split already exists in output h5 file.
2018-03-06 23:29:39,754 [MainThread  ] [WARNI]  Skipping oocytes_merluccius_states_2f as test/train split already exists in output h5 file.
2018-03-06 23:29:39,779 [MainThread  ] [WARNI]  Skipping oocytes_trisopterus_nucleus_2f as test/train split already exists in output h5 file.
2018-03-06 23:29:39,799 [MainThread  ] [WARNI]  Skipping oocytes_trisopterus_states_5b as test/train split already exists in output h5 file.
2018-03-06 23:29:39,819 [MainThread  ] [WARNI]  Skipping optical as test/train split already exists in output h5 file.
2018-03-06 23:29:39,838 [MainThread  ] [WARNI]  Skipping ozone as test/train split already exists in output h5 file.
2018-03-06 23:29:39,871 [MainThread  ] [WARNI]  Skipping page_blocks as test/trai

### Step 5: Run the experiments

In [6]:
test_o.run(input_io_datasets_loc=dts_names_list_full_path[0:3], output_io_split_idx_loc=split_dts_list[0:3], modelling_strategies=instantiated_models)

2018-01-27 15:09:26,093 [MainThread  ] [WARNI]  Estimator LogisticRegression already trained on abalone. Loading it from disk.
2018-01-27 15:09:26,093 [MainThread  ] [WARNI]  Estimator LogisticRegression already trained on abalone. Loading it from disk.
2018-01-27 15:09:26,096 [MainThread  ] [WARNI]  Estimator RidgeRegression already trained on abalone. Loading it from disk.
2018-01-27 15:09:26,096 [MainThread  ] [WARNI]  Estimator RidgeRegression already trained on abalone. Loading it from disk.
2018-01-27 15:09:26,099 [MainThread  ] [WARNI]  Estimator Lasso already trained on abalone. Loading it from disk.
2018-01-27 15:09:26,099 [MainThread  ] [WARNI]  Estimator Lasso already trained on abalone. Loading it from disk.
2018-01-27 15:09:26,102 [MainThread  ] [WARNI]  Estimator LassoLars already trained on abalone. Loading it from disk.
2018-01-27 15:09:26,102 [MainThread  ] [WARNI]  Estimator LassoLars already trained on abalone. Loading it from disk.
2018-01-27 15:09:26,107 [MainThrea

*** Training models on dataset: abalone. Total datasets processed: 0/3 ***
*** Training models on dataset: acute_inflammation. Total datasets processed: 1/3 ***
*** Training models on dataset: acute_nephritis. Total datasets processed: 2/3 ***


### Step 6: Analyze the results

In [7]:
analyze = AnalyseResults(hdf5_output_io=out_io, hdf5_input_io=input_io)
observations = analyze.calculate_loss_all_datasets(input_h5_original_datasets_group='delgado_datasets/', 
                                    output_h5_predictions_group='experiments/predictions/', 
                                    metric='mean_squared_error')

#### t-test

In [8]:
t_test, t_test_df = analyze.t_test(observations)
t_test_df

,pair,t_statistic,p_value
0,Lasso - LassoLars,-0.177322,0.859405
1,Lasso - LogisticRegression,0.411066,0.681391
2,Lasso - PassiveAggressiveClassifier,-0.253793,0.799873
3,Lasso - RidgeRegression,-0.011886,0.990526
4,LassoLars - LogisticRegression,0.605498,0.545420
5,LassoLars - PassiveAggressiveClassifier,-0.097207,0.922643
6,LassoLars - RidgeRegression,0.164864,0.869190
7,LogisticRegression - PassiveAggressiveClassifier,-0.623414,0.533605
8,LogisticRegression - RidgeRegression,-0.422386,0.673121
9,PassiveAggressiveClassifier - RidgeRegression,0.242680,0.808461


#### sign test

In [9]:
sign_test, sign_test_df = analyze.sign_test(observations)
sign_test_df

,pair,t_statistic,p_value
0,Lasso - LassoLars,0.901745,0.367192
1,Lasso - LogisticRegression,0.087236,0.930484
2,Lasso - PassiveAggressiveClassifier,-0.259871,0.794963
3,Lasso - RidgeRegression,1.018365,0.308504
4,LassoLars - LogisticRegression,-0.508724,0.610946
5,LassoLars - PassiveAggressiveClassifier,-0.832874,0.404916
6,LassoLars - RidgeRegression,0.123967,0.901341
7,LogisticRegression - PassiveAggressiveClassifier,-0.403122,0.686858
8,LogisticRegression - RidgeRegression,0.641873,0.520955
9,PassiveAggressiveClassifier - RidgeRegression,0.955923,0.339111


#### t-test with bonferroni correction

In [10]:
t_test_bonferroni, t_test_bonferroni_df = analyze.t_test_with_bonferroni_correction(observations)
t_test_bonferroni_df

,pair,p_value
0,Lasso - LassoLars,1.0
1,Lasso - LogisticRegression,1.0
2,Lasso - PassiveAggressiveClassifier,1.0
3,Lasso - RidgeRegression,1.0
4,LassoLars - LogisticRegression,1.0
5,LassoLars - PassiveAggressiveClassifier,1.0
6,LassoLars - RidgeRegression,1.0
7,LogisticRegression - PassiveAggressiveClassifier,1.0
8,LogisticRegression - RidgeRegression,1.0
9,PassiveAggressiveClassifier - RidgeRegression,1.0


#### Wilcoxon test

In [11]:
wilcoxon_test, wilcoxon_test_df = analyze.wilcoxon_test(observations)
wilcoxon_test_df

,pair,statistic,p_value
0,Lasso - LassoLars,1520.0,2.470469e-07
1,Lasso - LogisticRegression,3535.0,6.875244e-01
2,Lasso - PassiveAggressiveClassifier,2364.0,6.010731e-04
3,Lasso - RidgeRegression,2018.0,1.517763e-05
4,LassoLars - LogisticRegression,3064.0,1.051234e-01
5,LassoLars - PassiveAggressiveClassifier,1947.0,6.490607e-06
6,LassoLars - RidgeRegression,3249.0,2.534600e-01
7,LogisticRegression - PassiveAggressiveClassifier,1190.0,1.197243e-05
8,LogisticRegression - RidgeRegression,2969.0,6.200856e-02
9,PassiveAggressiveClassifier - RidgeRegression,1971.0,8.680879e-06


#### Friedman test

In [12]:
friedman_test, friedman_test_df = analyze.friedman_test(observations)
friedman_test_df

,statistic,p_value
0,74.35786,2.724079e-15


#### Nemenyi test

In [13]:
nemeniy_test = analyze.nemenyi(observations)
nemeniy_test

array([[-1.        ,  0.96311003,  0.99972355,  0.99995087,  0.93743453],
       [ 0.96311003, -1.        ,  0.98917175,  0.93329701,  0.99997031],
       [ 0.99972355,  0.98917175, -1.        ,  0.99801612,  0.97694221],
       [ 0.99995087,  0.93329701,  0.99801612, -1.        ,  0.89728878],
       [ 0.93743453,  0.99997031,  0.97694221,  0.89728878, -1.        ]])